In [ ]:
import pandas as pd 
# 1. wczytajmy data frame z danymi "mieszkania.csv" (via pandas)
# TWÓJ KOD: 

In [ ]:
# 2. rzućmy okiem na pierwszych 10 wierszy dataframe'u [https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html]
# TWÓJ KOD: 

In [ ]:
# Rzućmy okiem na rozkłady (histogramy) ceny [https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.hist.html] 
# TWÓJ KOD:

In [ ]:
# 4. W jaki sposób cena zależy od powierzchni [https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html] 
# użyj parametrów `marker` i `kind`
# TWÓJ KOD:

### 3. Definicja modelu 
#### Uproszczony "zabawkowy" model liniowy 
$$
cena = w_{\text{wiek}} * x_{\text{wiek}} + w_{\text{powierzchnia}} * x_{\text{powierzchnia}}
$$

In [ ]:
# Zdefiniujmy Grid parametrów (zakładamy wiedzę domenową) 

from tqdm import tqdm 
import matplotlib.pyplot as plt

GRID = {
    'powierzchnia': list(range(8000, 25000, 2000)), 
    'wiek': list(range(-25000, 1000, 500))
}

def plot_grid():
    points = []
    
    for i, w_wiek_i in enumerate(GRID['wiek']):
        for j, w_powierzchnia_j in enumerate(GRID['powierzchnia']):            
            # TODO: 
            pass     

    x = [point[0] for point in points]
    y = [point[1] for point in points]
    plt.figure(figsize=(10,5))

    plt.scatter(x, y, marker='.')

    plt.xlabel('parametr wiek')
    plt.ylabel('parametr powierzchnia')


In [ ]:
plot_grid()

In [ ]:
from typing import List 

def compute_mse(predicted_y_cena: pd.Series, y_cena: pd.Series) -> float:
    # TODO: 
    
class ToyLinearModel:

    def __init__(self, weight_wiek = 0, weight_powierzchnia = 0):
        self.weight_wiek = weight_wiek 
        self.weight_powierzchnia = weight_powierzchnia

    def predict(self, wiek: pd.Series, powierzchnia: pd.Series) -> pd.Series:
        # TODO 

    def train(self, x_wiek: pd.Series, x_powierzchnia: pd.Series, y_cena: pd.Series):
        assert len(x_wiek) == len(x_powierzchnia) == len(y_cena)
        best_mse = np.inf
        best_weight_wiek = 0 
        best_weight_powierzchnia = 0
        mses = []
        for weight_powierzchnia in tqdm(GRID['powierzchnia']):
            for weight_wiek in GRID['wiek']:
                self.weight_wiek = weight_wiek
                self.weight_powierzchnia = weight_powierzchnia
                predicted_y_cena = 0 # TODO 
                current_mse = 0 # TODO 
                if best_mse > current_mse:
                    best_weight_wiek = 0 # TODO 
                    best_weight_powierzchnia = 0 # TODO 
                    best_mse = 0 # TODO
                    mses.append(best_mse)
        print(f"Found best params to be {best_weight_wiek=} and {best_weight_powierzchnia=}")
        self.weight_wiek = best_weight_wiek
        self.weight_powierzchnia = best_weight_powierzchnia
        return mses 
        

In [ ]:
model = ToyLinearModel()

In [ ]:
mse_list = model.train(house_prices.wiek, house_prices.powierzchnia, house_prices.cena_pln)

In [ ]:
# Zobaczmy jak wyglądał postęp minimalizowania mse (plot mse_list) 
# TODO: 

In [ ]:
predicted_prices = house_prices.apply(lambda x : model.predict(x.wiek, x.powierzchnia), axis=1)
# dodajmy kolumne "predykowana_cena_pln" [https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.assign.html]
# TODO: 

In [ ]:
# Zobaczmy jak blisko jesteśmy "idealnego" dopasowania
ax = house_prices.plot(your='code') # TODO  
ax.axline((0,0), slope=1, color='black')

In [ ]:
# Wyznaczmy predykowaną cenę 10 letniego, 50 metrowego mieszkania
# TODO: 